# Advent of Code 2018

In [62]:
import os
import re

from collections import Counter, defaultdict
from datetime import datetime
from itertools import accumulate, combinations, cycle

import numpy as np
import pandas as pd
import requests

In [2]:
def input_for_day(day):
    file_name = f"day{day:0>2}_input"
    if os.path.exists(file_name):
        return os.path.abspath(file_name)
    with open("session", "r") as session_file:
        response = requests.get(f"https://adventofcode.com/2018/day/{day}/input",
                                cookies={"session": session_file.read()})
    with open(file_name, "w") as input_file:
        print(response.text.strip(), file=input_file)
    return os.path.abspath(file_name)

## Day 1
### Part One
Starting with a frequency of zero, what is the resulting frequency after all of the changes in frequency have been applied?

In [3]:
%time
with open(input_for_day(1), "r") as input_file:
    print(f'Answer: {sum(map(int, input_file))}')

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 9.54 µs
Answer: 538


### Part Two
What is the first frequency your device reaches twice?

In [4]:
%%time
freq_set = {0}
with open(input_for_day(1), "r") as input_file:
    for freq in accumulate(cycle(int(line) for line in input_file)):
        if freq in freq_set or freq_set.add(freq):
            print(f"Answer: {freq}")
            break

Answer: 77271
CPU times: user 50.5 ms, sys: 20.1 ms, total: 70.5 ms
Wall time: 86.7 ms


## Day 2
### Part One
What is the checksum for your list of box IDs?

In [5]:
%%time
with open(input_for_day(2), "r") as input_file:
    counters = [Counter(n.strip()).values() for n in input_file]
checksum = sum(2 in c for c in counters) * sum(3 in c for c in counters)
print(f"Answer: {checksum}")

Answer: 5976
CPU times: user 10.3 ms, sys: 321 µs, total: 10.7 ms
Wall time: 154 ms


### Part Two
What letters are common between the two correct box IDs?

In [6]:
%%time
with open(input_for_day(2), "r") as input_file:
    counters = {x: Counter(x).values() for x in (n.strip() for n in input_file)}
for x, y in combinations((n for n, c in counters.items() if any(s in c for s in (2, 3))), 2):
    if sum(a != b for a, b in zip(x, y)) == 1:
        print(f"Answer: {''.join(a for a, b in zip(x, y) if a == b)}")
        break

Answer: xretqmmonskvzupalfiwhcfdb
CPU times: user 21.2 ms, sys: 676 µs, total: 21.8 ms
Wall time: 37.1 ms


## Day 3
### Part One
How many square inches of fabric are within two or more claims?

In [7]:
%%time
expr = re.compile(r"#(\d+)\s@\s(\d+),(\d+):\s(\d+)x(\d+)")
fabric = np.zeros((1000, 1000), dtype=np.uint32)
with open(input_for_day(3), "r") as input_file:
    for claim in input_file:
        _, x, y, w, h = map(int, expr.match(claim).groups())
        fabric[y:y+h, x:x+w] += 1
    print(f"Answer: {np.sum(fabric > 1)}")

Answer: 117948
CPU times: user 23.4 ms, sys: 0 ns, total: 23.4 ms
Wall time: 127 ms


### Part Two
What is the ID of the only claim that doesn't overlap?

In [8]:
%%time
expr = re.compile(r"#(\d+)\s@\s(\d+),(\d+):\s(\d+)x(\d+)")
fabric = np.zeros((1000, 1000), dtype=np.uint32)
with open(input_for_day(3), "r") as input_file:
    claims = [list(map(int, expr.match(claim).groups())) for claim in input_file]
for claim in claims:
    _, x, y, w, h = claim
    fabric[y:y+h, x:x+w] += 1
for claim in claims:
    claim_id, x, y, w, h = claim
    if np.all(fabric[y:y+h, x:x+w] == 1):
        print(f"Answer: {claim_id}")
        break

Answer: 567
CPU times: user 19.4 ms, sys: 9.32 ms, total: 28.7 ms
Wall time: 54.3 ms


## Day 4
### Part One
What is the ID of the guard you chose multiplied by the minute you chose?

In [113]:
%%time
entry_expr = re.compile(r"\[(.+)\]\s(.*)")
guard_expr = re.compile("Guard\s#(\d+)")

guards = defaultdict(Counter)
with open(input_for_day(4), "r") as input_file:
    lines = (entry_expr.match(entry).groups() for entry in sorted(input_file))
for timestamp, event in ((datetime.strptime(t, "%Y-%m-%d %H:%M"), e) for t, e in lines):
    if event.startswith("Guard"):
        guard = int(guard_expr.match(event).group(1))
    elif event.startswith("falls"):
        start = timestamp
    elif event.startswith("wakes"):
        duration = int((timestamp - start).total_seconds()) // 60
        guards[guard].update(Counter(start.minute + i for i in range(duration)))

_, guard = max((sum(c.values()), g) for g, c in guards.items())

print(f"Answer: {guard * guards[guard].most_common()[0][0]}")

Answer: 4716
CPU times: user 32 ms, sys: 1.84 ms, total: 33.8 ms
Wall time: 71 ms


### Part Two
What is the ID of the guard you chose multiplied by the minute you chose? 

In [112]:
%%time
entry_expr = re.compile(r"\[(.+)\]\s(.*)")
guard_expr = re.compile("Guard\s#(\d+)")

guards = defaultdict(Counter)
with open(input_for_day(4), "r") as input_file:
    lines = (entry_expr.match(entry).groups() for entry in sorted(input_file))
for timestamp, event in ((datetime.strptime(t, "%Y-%m-%d %H:%M"), e) for t, e in lines):
    if event.startswith("Guard"):
        guard = int(guard_expr.match(event).group(1))
    elif event.startswith("falls"):
        start = timestamp
    elif event.startswith("wakes"):
        duration = int((timestamp - start).total_seconds()) // 60
        guards[guard].update(Counter(start.minute + i for i in range(duration)))

(_, minute), guard = max((c.most_common()[0][::-1], g) for g, c in guards.items())

print(f"Answer: {guard * minute}")

Answer: 117061
CPU times: user 31.1 ms, sys: 0 ns, total: 31.1 ms
Wall time: 77.9 ms
